In [1]:
%load_ext autoreload
%autoreload 2



## Generate Initial Encoding for User and Movie

In [2]:
import torch
import os
import numpy as np
torch.load("genres_hot.pt").size()

C:\Users\xiaoyicong\AppData\Local\Temp\ipykernel_14392\1724816224.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load("genres_hot.pt").size()


torch.Size([9742, 20])

In [3]:
from models.clus_gcn_model import ClusGCN, ClusGCNModel

model_wrapper = ClusGCNModel(
        size="100k",
        embedding_dim=64,
        num_layers=5,
        learning_rate=0.01,
        epochs=50,
        device='cpu'
    )

model_wrapper.prepare_training_data()

ages2code = {"Under 18":0, "18-24":1, "25-34":2, "35-44":3, "45-49":4, "50-55":5, "56+":6}
occupations2code = {
        "administrator": 0,
        "artist": 1,
        "doctor": 2,
        "educator": 3,
        "engineer": 4,
        "entertainment": 5,
        "executive": 6,
        "healthcare": 7,
        "homemaker": 8,
        "lawyer": 9,
        "librarian": 10,
        "marketing": 11,
        "none": 12,
        "other": 13,
        "programmer": 14,
        "retired": 15,
        "salesman": 16,
        "scientist": 17,
        "student": 18,
        "technician": 19,
        "writer": 20
    }



2024-11-07 16:17:27,140 - ClusGCN model - [INFO]: Using CPU
2024-11-07 16:17:27,142 - ClusGCN model - [INFO]: Preparing data...
2024-11-07 16:17:27,537 - ClusGCN model - [INFO]: Data Loaded.
2024-11-07 16:17:28,078 - ClusGCN model - [INFO]: Generating negative samples...
2024-11-07 16:17:28,711 - ClusGCN model - [INFO]: Generating negative samples...
2024-11-07 16:17:28,851 - ClusGCN model - [INFO]: building graph...
2024-11-07 16:17:28,877 - ClusGCN model - [INFO]: initilizing model...
d:\NUS\CS5284_graph_ml\project\dinghui101\models\clus_gcn_model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This l

['c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\python310.zip', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\DLLs', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101', '', 'C:\\Users\\xiaoyicong\\AppData\\Roaming\\Python\\Python310\\site-packages', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages\\win32', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages\\win32\\lib', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages\\Pythonwin', 'd:\\NUS\\CS5284_graph_ml\\project\\dinghui101']


In [4]:
model_wrapper.train_df

,user_idx,item_idx,rating,label
0,0,0,3,1
1,0,528,4,1
2,0,377,4,1
3,0,522,3,1
4,0,431,5,1
...,...,...,...,...
495280,232,771,0,0
495281,232,1433,0,0
495282,232,1084,0,0
495283,232,1275,0,0


In [5]:
model_wrapper.test_df["label"].describe()

count    4715.000000
mean        0.200000
std         0.400042
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [6]:
model_wrapper.train_df["label"].describe()

count    495285.0
mean          0.2
std           0.4
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           1.0
Name: label, dtype: float64

In [7]:
495285/5, 4715/5

(99057.0, 943.0)

In [8]:
model_wrapper.items

AttributeError: 'ClusGCNModel' object has no attribute 'items'

In [ ]:
# sort the row order of the movies
# the order follows the "index" instead of the movie_id
model_wrapper.items['sorting_index'] = model_wrapper.items['movie_id'].map(model_wrapper.item2idx)
model_wrapper.items = model_wrapper.items.sort_values('sorting_index').drop(columns='sorting_index').reset_index(drop=True)

In [ ]:
model_wrapper.items

,movie_id,title,year,genre_unknown,genre_Action,genre_Adventure,genre_Animation,genre_Children's,genre_Comedy,genre_Crime,...,genre_Fantasy,genre_Film-Noir,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,242,Kolya,1996,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,302,L.A. Confidential,1997,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,377,Heavyweights,1994,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,51,Legends of the Fall,1994,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,346,Jackie Brown,1997,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1674,Mamma Roma,1962,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1640,"Eighth Day, The",1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1679,1637,Girls Town,1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1680,1630,"Silence of the Palace, The (Saimt el Qusur)",1994,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
genre_columns = model_wrapper.items.columns[-19:]
genre_columns

Index(['genre_unknown', 'genre_Action', 'genre_Adventure', 'genre_Animation',
       'genre_Children's', 'genre_Comedy', 'genre_Crime', 'genre_Documentary',
       'genre_Drama', 'genre_Fantasy', 'genre_Film-Noir', 'genre_Horror',
       'genre_Musical', 'genre_Mystery', 'genre_Romance', 'genre_Sci-Fi',
       'genre_Thriller', 'genre_War', 'genre_Western'],
      dtype='object')

In [ ]:
genre_values = model_wrapper.items[genre_columns].values
genre_tensor = torch.tensor(genre_values, dtype=torch.float)

In [ ]:
print(genre_tensor.shape)
torch.save(genre_tensor, 'genre_hot_embeddings.pt')

torch.Size([1682, 19])


In [ ]:
model_wrapper.user_features

,userid,age,gender,occupation,zipcode
0,1,18-24,M,technician,85711
1,2,50-55,F,other,94043
2,3,18-24,M,writer,32067
3,4,18-24,M,technician,43537
4,5,25-34,F,other,15213
...,...,...,...,...,...
938,939,25-34,F,student,33319
939,940,25-34,M,administrator,02215
940,941,18-24,M,student,97229
941,942,45-49,F,librarian,78209


In [ ]:
# Users
model_wrapper.user_features['sorting_index'] = model_wrapper.user_features['userid'].map(model_wrapper.user2idx)
model_wrapper.user_features = model_wrapper.user_features.sort_values('sorting_index').drop(columns='sorting_index').reset_index(drop=True)

# Generate one-hot encoding for age
age_encoding = torch.zeros(len(model_wrapper.user_features), len(ages2code))
for i, age_range in enumerate(model_wrapper.user_features['age']):
    age_encoding[i][ages2code[age_range]] = 1

# Generate one-hot encoding for occupation
occupation_encoding = torch.zeros(len(model_wrapper.user_features), len(occupations2code))
for i, occupation in enumerate(model_wrapper.user_features['occupation']):
    occupation_encoding[i][occupations2code[occupation]] = 1

# Save the encodings to .pt files
torch.save(age_encoding, 'user_age_hot_embedding.pt')
torch.save(occupation_encoding, 'user_occupation_hot_embedding.pt')

In [ ]:
age_encoding.size(), occupation_encoding.size()

(torch.Size([943, 7]), torch.Size([943, 21]))

In [ ]:
grouped = model_wrapper.ratings.groupby('user_idx')

for user, group in grouped:
    #print(user)
    #print(group)
    test_sample = group.sample(n=1, random_state=42)
    train_sample = group.drop(test_sample.index)
    print(test_sample)
    print(train_sample)
    break
    # if len(group) < 2:
    #     train_list.append(group)
    # else:
    #     test_sample = group.sample(n=1, random_state=42)
    #     train_sample = group.drop(test_sample.index)
    #     train_list.append(train_sample)
    #     test_list.append(test_sample)

       user  item  rating  timestamp  user_idx  item_idx
59165   196   762       3  881251955         0       365
       user  item  rating  timestamp  user_idx  item_idx
0       196   242       3  881250949         0         0
940     196   393       4  881251863         0       528
1133    196   381       4  881251728         0       377
1812    196   251       3  881251274         0       522
1896    196   655       5  881251793         0       431
2374    196    67       5  881252017         0       834
6910    196   306       4  881251021         0       380
7517    196   238       4  881251820         0       329
7842    196   663       5  881251911         0       550
10017   196   111       4  881251793         0        83
10254   196   580       2  881252056         0       632
10981   196    25       4  881251955         0        86
13733   196   286       5  881250949         0       289
14606   196    94       3  881252172         0       363
16834   196   692       5  8812

## Load Model Wrapper

In [ ]:
from models.clus_gcn_model import ClusGCN, ClusGCNModel

model_wrapper = ClusGCNModel(
        size="100k",
        embedding_dim=64,
        num_layers=5,
        learning_rate=0.01,
        epochs=50,
        device='cpu'
    )

model_wrapper.prepare_training_data()

2024-11-01 01:58:02,363 - LGCN model - [INFO]: Using CPU
2024-11-01 01:58:02,369 - LGCN model - [INFO]: Preparing data...
2024-11-01 01:58:02,731 - LGCN model - [INFO]: Data Loaded.


   user  item  rating  timestamp
0   196   242       3  881250949
1   186   302       3  891717742
2    22   377       1  878887116
3   244    51       2  880606923
4   166   346       1  886397596


2024-11-01 01:58:03,250 - LGCN model - [INFO]: Generating negative samples...
2024-11-01 01:58:03,789 - LGCN model - [INFO]: Generating negative samples...
2024-11-01 01:58:03,922 - LGCN model - [INFO]: building graph...
2024-11-01 01:58:03,944 - LGCN model - [INFO]: initilizing model...


adj_norm: tensor(indices=tensor([[   0,    0,    0,  ..., 2622, 2623, 2624],
                       [ 943,  953, 1026,  ...,  650,  650,  650]]),
       values=tensor([0.0151, 0.0094, 0.0099,  ..., 0.0382, 0.0382, 0.0382]),
       size=(2625, 2625), nnz=198114, layout=torch.sparse_coo)
['c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\python310.zip', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\DLLs', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101', '', 'C:\\Users\\xiaoyicong\\AppData\\Roaming\\Python\\Python310\\site-packages', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages\\win32', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages\\win32\\lib', 'c:\\Users\\xiaoyicong\\anaconda3\\envs\\dinghui101\\lib\\site-packages\\Pythonwin', 'd:\\NUS\\CS5284_graph_ml\\project\\dinghui101']


d:\NUS\CS5284_graph_ml\project\dinghui101\models\clus_gcn_model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.genre_hot_embedding = torch.load(os.path.join(embed

In [ ]:
model_wrapper.model

ClusGCN(
  (user_embedding): Embedding(943, 64)
  (item_embedding): Embedding(1682, 64)
  (genre_cat_layer): Linear(in_features=19, out_features=64, bias=True)
  (age_cat_layer): Linear(in_features=7, out_features=64, bias=True)
  (occupation_cat_layer): Linear(in_features=21, out_features=64, bias=True)
)

## Clustering step

In [9]:
model = model_wrapper.model
all_embeddings = torch.cat(
            [model.user_embedding.weight, 
             model.item_embedding.weight], dim=0
        )
model.user_embedding.weight.size(), model.item_embedding.weight.size(), all_embeddings.size()

(torch.Size([943, 64]), torch.Size([1682, 64]), torch.Size([2625, 64]))

In [10]:
model.user_embedding

Embedding(943, 64)

In [55]:
user_degree = {user_idx: model_wrapper.adj_norm[user_idx]._nnz() for user_idx in range(model_wrapper.num_users)}
user_degree

{0: 38,
 1: 91,
 2: 127,
 3: 237,
 4: 19,
 5: 126,
 6: 91,
 7: 96,
 8: 221,
 9: 210,
 10: 231,
 11: 287,
 12: 215,
 13: 131,
 14: 140,
 15: 483,
 16: 60,
 17: 304,
 18: 295,
 19: 479,
 20: 180,
 21: 68,
 22: 279,
 23: 396,
 24: 277,
 25: 120,
 26: 215,
 27: 92,
 28: 119,
 29: 23,
 30: 274,
 31: 26,
 32: 149,
 33: 62,
 34: 50,
 35: 434,
 36: 22,
 37: 517,
 38: 402,
 39: 183,
 40: 46,
 41: 385,
 42: 61,
 43: 194,
 44: 19,
 45: 160,
 46: 135,
 47: 272,
 48: 76,
 49: 57,
 50: 23,
 51: 77,
 52: 381,
 53: 136,
 54: 210,
 55: 182,
 56: 123,
 57: 47,
 58: 635,
 59: 50,
 60: 207,
 61: 105,
 62: 104,
 63: 186,
 64: 80,
 65: 387,
 66: 22,
 67: 158,
 68: 387,
 69: 22,
 70: 386,
 71: 184,
 72: 180,
 73: 58,
 74: 41,
 75: 433,
 76: 315,
 77: 78,
 78: 54,
 79: 40,
 80: 299,
 81: 130,
 82: 121,
 83: 277,
 84: 45,
 85: 180,
 86: 68,
 87: 132,
 88: 153,
 89: 47,
 90: 399,
 91: 183,
 92: 150,
 93: 119,
 94: 51,
 95: 167,
 96: 159,
 97: 176,
 98: 220,
 99: 67,
 100: 322,
 101: 45,
 102: 287,
 103: 125,
 1

In [65]:
user_degree = {user_idx: model_wrapper.adj_norm[user_idx]._nnz() for user_idx in range(model_wrapper.num_users)}
user_degree_ts = torch.tensor(list(user_degree.values()))

In [ ]:
torch.min(1, user_degree_ts / torch.quantile(user_degree_ts.float(), 0.75))

tensor([0.2585, 0.6190, 0.8639, 1.6122, 0.1293, 0.8571, 0.6190, 0.6531, 1.5034,
        1.4286, 1.5714, 1.9524, 1.4626, 0.8912, 0.9524, 3.2857, 0.4082, 2.0680,
        2.0068, 3.2585, 1.2245, 0.4626, 1.8980, 2.6939, 1.8844, 0.8163, 1.4626,
        0.6259, 0.8095, 0.1565, 1.8639, 0.1769, 1.0136, 0.4218, 0.3401, 2.9524,
        0.1497, 3.5170, 2.7347, 1.2449, 0.3129, 2.6190, 0.4150, 1.3197, 0.1293,
        1.0884, 0.9184, 1.8503, 0.5170, 0.3878, 0.1565, 0.5238, 2.5918, 0.9252,
        1.4286, 1.2381, 0.8367, 0.3197, 4.3197, 0.3401, 1.4082, 0.7143, 0.7075,
        1.2653, 0.5442, 2.6327, 0.1497, 1.0748, 2.6327, 0.1497, 2.6259, 1.2517,
        1.2245, 0.3946, 0.2789, 2.9456, 2.1429, 0.5306, 0.3673, 0.2721, 2.0340,
        0.8844, 0.8231, 1.8844, 0.3061, 1.2245, 0.4626, 0.8980, 1.0408, 0.3197,
        2.7143, 1.2449, 1.0204, 0.8095, 0.3469, 1.1361, 1.0816, 1.1973, 1.4966,
        0.4558, 2.1905, 0.3061, 1.9524, 0.8503, 0.4966, 1.4558, 0.1429, 0.2245,
        0.1769, 0.1293, 2.2245, 1.1837, 

In [62]:
min(user_degree.values())

19

In [ ]:
from sklearn.cluster import KMeans

user_degree = {user_idx: model_wrapper.adj_norm[user_idx]._nnz() for user_idx in range(model_wrapper.num_users)}
user_degree_ts = torch.tensor(list(user_degree.values()))
user_degree_ts = user_degree_ts / torch.max(user_degree_ts)

device = "cpu"


# Assume model.user_embedding.weight contains the embeddings of data points
user_embedding = model.user_embedding.weight

# Convert embeddings to numpy array for KMeans clustering
user_embedding_np = user_embedding.cpu().detach().numpy()

# Apply KMeans clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters)
cluster_assignments = kmeans.fit_predict(user_embedding_np)

# Move cluster assignments back to torch tensor on GPU
cluster_assignments_tensor = torch.from_numpy(cluster_assignments).to(device)

# Compute the sum of embeddings for each cluster using matrix operations
cluster_sums = torch.zeros(num_clusters, user_embedding.size(1), device=device)
cluster_counts = torch.zeros(num_clusters, device=device)

# Calculate the sum of embeddings for each cluster using matrix operations
for i in range(user_embedding.size(0)):
    cluster_id = cluster_assignments_tensor[i]
    cluster_sums[cluster_id] += user_embedding[i]
    cluster_counts[cluster_id] += 1

# Compute the transformed embeddings
alphas = user_degree_ts.unsqueeze(1)
transformed_embeddings = alphas * user_embedding + (1-alphas)*cluster_sums[cluster_assignments_tensor] / cluster_counts[cluster_assignments_tensor].unsqueeze(1)

transformed_embeddings

c:\Users\xiaoyicong\anaconda3\envs\dinghui101\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


tensor([[ 0.1910, -0.6757,  0.3772,  ..., -0.2097, -0.5178, -0.1038],
        [-0.0419, -0.3186, -0.1524,  ..., -0.4141, -0.2935, -0.3454],
        [ 0.2559, -0.6586, -0.0416,  ..., -0.0947, -0.5442, -0.5491],
        ...,
        [ 0.2413, -0.6668, -0.0095,  ..., -0.0990, -0.5253, -0.5195],
        [ 0.2623, -0.6845, -0.0278,  ..., -0.0842, -0.5453, -0.5355],
        [-0.0223, -0.6105, -0.1491,  ...,  0.0416,  0.1750, -0.3519]],
       grad_fn=<AddBackward0>)

In [ ]:
user_embedding

Parameter containing:
tensor([[ 0.0399, -0.6872,  0.2115,  ..., -0.1991, -0.5630, -0.1116],
        [ 0.0414, -0.1972, -0.2142,  ..., -0.5335, -0.0364, -0.5535],
        [ 0.2428, -0.5653, -0.1471,  ..., -0.1545, -0.5591, -0.6355],
        ...,
        [ 0.0527, -0.5441,  0.1013,  ..., -0.2812, -0.3534, -0.3933],
        [ 0.3678, -0.8689, -0.2592,  ..., -0.1397, -0.6673, -0.6609],
        [ 0.0491, -0.6234, -0.1508,  ...,  0.0828,  0.1856, -0.4175]],
       requires_grad=True)

In [ ]:
alphas = torch.ones(user_embedding.size(0)).unsqueeze(1)
alphas, 1-alphas

(tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [

In [ ]:
user_embedding * alphas

tensor([[ 0.0399, -0.6872,  0.2115,  ..., -0.1991, -0.5630, -0.1116],
        [ 0.0414, -0.1972, -0.2142,  ..., -0.5335, -0.0364, -0.5535],
        [ 0.2428, -0.5653, -0.1471,  ..., -0.1545, -0.5591, -0.6355],
        ...,
        [ 0.0527, -0.5441,  0.1013,  ..., -0.2812, -0.3534, -0.3933],
        [ 0.3678, -0.8689, -0.2592,  ..., -0.1397, -0.6673, -0.6609],
        [ 0.0491, -0.6234, -0.1508,  ...,  0.0828,  0.1856, -0.4175]],
       grad_fn=<MulBackward0>)

In [19]:
model_wrapper.adj_norm

tensor(indices=tensor([[   0,    0,    0,  ..., 2622, 2623, 2624],
                       [ 943,  953, 1026,  ...,  650,  650,  650]]),
       values=tensor([0.0151, 0.0094, 0.0099,  ..., 0.0382, 0.0382, 0.0382]),
       size=(2625, 2625), nnz=198114, layout=torch.sparse_coo)

In [21]:
len(model_wrapper.adj_norm.values())

198114

In [26]:
model_wrapper.train_df['label'].sum()

99057

In [35]:
len(model_wrapper.train_df['user_idx'].unique())

943

In [36]:
1682 + 943

2625

In [37]:
model_wrapper.train_df

,user_idx,item_idx,rating,label
0,0,0,3,1
1,0,528,4,1
2,0,377,4,1
3,0,522,3,1
4,0,431,5,1
...,...,...,...,...
495280,232,771,0,0
495281,232,1433,0,0
495282,232,1084,0,0
495283,232,1275,0,0


In [40]:
df = model_wrapper.train_df[model_wrapper.train_df['label'] == 1]

In [41]:
df[df["user_idx"] == 1]

,user_idx,item_idx,rating,label
38,1,1,3,1
39,1,476,5,1
40,1,305,1,1
41,1,577,4,1
42,1,627,3,1
...,...,...,...,...
124,1,172,5,1
125,1,136,3,1
126,1,452,4,1
127,1,446,2,1


In [44]:
df[df["item_idx"] == 1]

,user_idx,item_idx,rating,label
38,1,1,3,1
721,6,1,4,1
857,8,1,4,1
1202,9,1,4,1
1268,10,1,3,1
...,...,...,...,...
97496,923,1,4,1
97866,927,1,4,1
98372,931,1,4,1
98585,936,1,4,1


In [53]:
model_wrapper.adj_norm[1]._nnz()

91

In [63]:
user_degree_ts = torch.tensor(list(user_degree.values()))
user_degree_ts / torch.max(user_degree_ts)

tensor([0.0516, 0.1236, 0.1726, 0.3220, 0.0258, 0.1712, 0.1236, 0.1304, 0.3003,
        0.2853, 0.3139, 0.3899, 0.2921, 0.1780, 0.1902, 0.6562, 0.0815, 0.4130,
        0.4008, 0.6508, 0.2446, 0.0924, 0.3791, 0.5380, 0.3764, 0.1630, 0.2921,
        0.1250, 0.1617, 0.0312, 0.3723, 0.0353, 0.2024, 0.0842, 0.0679, 0.5897,
        0.0299, 0.7024, 0.5462, 0.2486, 0.0625, 0.5231, 0.0829, 0.2636, 0.0258,
        0.2174, 0.1834, 0.3696, 0.1033, 0.0774, 0.0312, 0.1046, 0.5177, 0.1848,
        0.2853, 0.2473, 0.1671, 0.0639, 0.8628, 0.0679, 0.2812, 0.1427, 0.1413,
        0.2527, 0.1087, 0.5258, 0.0299, 0.2147, 0.5258, 0.0299, 0.5245, 0.2500,
        0.2446, 0.0788, 0.0557, 0.5883, 0.4280, 0.1060, 0.0734, 0.0543, 0.4062,
        0.1766, 0.1644, 0.3764, 0.0611, 0.2446, 0.0924, 0.1793, 0.2079, 0.0639,
        0.5421, 0.2486, 0.2038, 0.1617, 0.0693, 0.2269, 0.2160, 0.2391, 0.2989,
        0.0910, 0.4375, 0.0611, 0.3899, 0.1698, 0.0992, 0.2908, 0.0285, 0.0448,
        0.0353, 0.0258, 0.4443, 0.2364, 

In [61]:
np.exp(38)

3.1855931757113756e+16

In [60]:
1/(1 + torch.exp(user_degree_ts)/10)

tensor([3.1391e-16, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.6028e-08, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.7565e-26, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 2.9375e-29, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0262e-09,
        0.0000e+00, 5.1091e-11, 0.0000e+00, 1.1851e-26, 1.9287e-21, 0.0000e+00,
        2.7895e-09, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0531e-19, 0.0000e+00,
        3.2213e-26, 0.0000e+00, 5.6028e-08, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        9.8542e-33, 1.7588e-24, 1.0262e-09, 3.6251e-33, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 3.8740e-20, 0.0000e+00, 1.9287e-21,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.8049e-34, 0.0000e+00,
        2.7895e-09, 0.0000e+00, 0.0000e+00, 2.7895e-09, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 6.4702e-25, 1.5629e-